In [1]:
# CNNで 「文字画像データセット（平仮名73文字版）」 を予測
# 前回出力した特徴量、正解を読み込んで開始

In [55]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

import os
import glob

import cv2


In [4]:
path = './dataset/hiragana73/'

# charfeatures.npy から入力
charfeatures_load = np.load(path + 'charfeatures.npy')

In [5]:
charfeatures_load.shape

(80000, 2304)

In [6]:
# files_df.csv 読み込み
files_df_load = pd.read_csv(path + 'files_df.csv')

In [7]:
files_df_load.head()

,Image,Labels
0,./dataset/hiragana73/U306F\1929_1452973_0089.png,は
1,./dataset/hiragana73/U306D\1914_908938_0044.png,ね
2,./dataset/hiragana73/U308F\1926_938757_0025.png,わ
3,./dataset/hiragana73/U308B\1918_1210785_0064.png,る
4,./dataset/hiragana73/U306F\1914_1239432_0055.png,は


In [8]:
files_df_load.describe()

,Image,Labels
count,80000,80000
unique,80000,73
top,./dataset/hiragana73/U306C\1912_899565_0170.png,し
freq,1,1285


In [10]:
files_df_load.size, files_df_load.shape

(160000, (80000, 2))

In [11]:
# ターゲット
y = files_df_load['Labels']

In [12]:
y.value_counts()

し    1285
そ    1285
や    1285
ん    1285
ま    1285
ふ    1285
た    1285
わ    1283
お    1283
す    1282
ゆ    1282
く    1266
さ    1261
に    1260
か    1259
を    1254
は    1247
り    1244
れ    1238
な    1233
め    1233
て    1213
あ    1208
が    1200
る    1190
も    1187
と    1184
で    1178
よ    1166
せ    1165
     ... 
ぜ    1115
こ    1115
へ    1114
ら    1114
べ    1109
ば    1105
じ    1099
ぬ    1093
ぎ    1091
づ    1080
ご    1078
え    1077
ひ    1074
ざ    1070
ろ    1069
ぞ    1066
げ    1058
む    1058
ゐ    1053
ち    1052
ず    1046
び    1045
ぼ    1044
ぐ    1043
ゑ    1030
ぺ     268
ぱ     262
ぽ     261
ぴ     126
ぷ     112
Name: Labels, Length: 73, dtype: int64

In [13]:
# ターゲットの平仮名を数値に変換
encoder = LabelEncoder()

encoder.fit(y)

LabelEncoder()

In [14]:
encoder.classes_

array(['あ', 'い', 'う', 'え', 'お', 'か', 'が', 'き', 'ぎ', 'く', 'ぐ', 'け', 'げ',
       'こ', 'ご', 'さ', 'ざ', 'し', 'じ', 'す', 'ず', 'せ', 'ぜ', 'そ', 'ぞ', 'た',
       'だ', 'ち', 'ぢ', 'つ', 'づ', 'て', 'で', 'と', 'ど', 'な', 'に', 'ぬ', 'ね',
       'の', 'は', 'ば', 'ぱ', 'ひ', 'び', 'ぴ', 'ふ', 'ぶ', 'ぷ', 'へ', 'べ', 'ぺ',
       'ほ', 'ぼ', 'ぽ', 'ま', 'み', 'む', 'め', 'も', 'や', 'ゆ', 'よ', 'ら', 'り',
       'る', 'れ', 'ろ', 'わ', 'ゐ', 'ゑ', 'を', 'ん'], dtype=object)

In [15]:
# 平仮名と数字を辞書型にする
encoder_character_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

In [17]:
print(encoder_character_mapping)

{'あ': 0, 'い': 1, 'う': 2, 'え': 3, 'お': 4, 'か': 5, 'が': 6, 'き': 7, 'ぎ': 8, 'く': 9, 'ぐ': 10, 'け': 11, 'げ': 12, 'こ': 13, 'ご': 14, 'さ': 15, 'ざ': 16, 'し': 17, 'じ': 18, 'す': 19, 'ず': 20, 'せ': 21, 'ぜ': 22, 'そ': 23, 'ぞ': 24, 'た': 25, 'だ': 26, 'ち': 27, 'ぢ': 28, 'つ': 29, 'づ': 30, 'て': 31, 'で': 32, 'と': 33, 'ど': 34, 'な': 35, 'に': 36, 'ぬ': 37, 'ね': 38, 'の': 39, 'は': 40, 'ば': 41, 'ぱ': 42, 'ひ': 43, 'び': 44, 'ぴ': 45, 'ふ': 46, 'ぶ': 47, 'ぷ': 48, 'へ': 49, 'べ': 50, 'ぺ': 51, 'ほ': 52, 'ぼ': 53, 'ぽ': 54, 'ま': 55, 'み': 56, 'む': 57, 'め': 58, 'も': 59, 'や': 60, 'ゆ': 61, 'よ': 62, 'ら': 63, 'り': 64, 'る': 65, 'れ': 66, 'ろ': 67, 'わ': 68, 'ゐ': 69, 'ゑ': 70, 'を': 71, 'ん': 72}


In [18]:
# 平仮名
print('---------平仮名---------')
y[:5]

---------平仮名---------


0    は
1    ね
2    わ
3    る
4    は
Name: Labels, dtype: object

In [19]:
# 変換した数値
y_encoder = encoder.transform(y)

print('---------数値---------')
y_encoder[:5]

---------数値---------


array([40, 38, 68, 65, 40])

In [20]:
# エンコードした値での確認
encoder_character_mapping['は']

40

In [22]:
# 訓練データとテストデータに分割
X_train, X_test, y_train, y_tset = train_test_split(charfeatures_load, y_encoder, train_size=10000, test_size=10000, random_state=42)

In [24]:
X_train.shape, y_train.shape, X_test.shape, y_tset.shape

((10000, 2304), (10000,), (10000, 2304), (10000,))

In [25]:
# 特徴量の正規化
# 割れないので float に変換」
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 正規化
X_train /= 255
X_test /= 255

In [26]:
# 形式変更
# バッチサイズ、(h 48) * (w 48) * (1 channel)
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [28]:
X_train.shape, X_test.shape

((10000, 48, 48, 1), (10000, 48, 48, 1))

In [34]:
np.unique(y_train).size

73

In [36]:
y_eval = y_tset.copy()

In [37]:
# カテゴリカル変数に変換
y_train = np_utils.to_categorical(y_train, np.unique(y_train).size)
y_tset = np_utils.to_categorical(y_tset, np.unique(y_tset).size)

In [40]:
y_tset[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.], dtype=float32)

In [39]:
y_eval[0]

19

In [43]:
np.argmax(y_tset[0])

19

In [45]:
np.argmax(y_tset[:5], axis=1)

array([19, 52, 27, 63, 18], dtype=int64)

In [46]:
# model_1
model_1 = Sequential()

# 入力層、畳み込み層
model_1.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=(48, 48, 1)))

Instructions for updating:
Colocations handled automatically by placer.


In [47]:
# プール層
model_1.add(MaxPooling2D(pool_size=(2, 2)))

In [50]:
# 畳み込み層とプール層
model_1.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))

model_1.add(MaxPooling2D(pool_size=(2, 2)))

In [51]:
# 全結合層、出力層
model_1.add(Flatten())

model_1.add(Dense(128, activation='relu'))

model_1.add(Dense(73, activation='softmax'))

In [53]:
# モデルコンパイル
model_1.compile(loss='categorical_crossentropy',
               optimizer='sgd',
               metrics=['accuracy'])

In [54]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_2 (Dense)              (None, 73)               

In [56]:
# EarkyStopping の設定
callbacks = EarlyStopping(monitor='val_loss', patience=3)

In [57]:
# モデル１で訓練
model_1.fit(X_train, y_train, validation_data=(X_test, y_tset), callbacks=[callbacks], epochs=50)

Instructions for updating:
Use tf.cast instead.
Train on 10000 samples, validate on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 31s 3ms/step - loss: 4.2603 - accuracy: 0.0255 - val_loss: 4.2234 - val_accuracy: 0.0361
Epoch 2/50
10000/10000 [==============================] - 26s 3ms/step - loss: 3.9925 - accuracy: 0.1032 - val_loss: 4.0200 - val_accuracy: 0.0558
Epoch 3/50
10000/10000 [==============================] - 26s 3ms/step - loss: 2.0778 - accuracy: 0.5016 - val_loss: 1.0191 - val_accuracy: 0.7368
Epoch 4/50
10000/10000 [==============================] - 26s 3ms/step - loss: 0.5810 - accuracy: 0.8725 - val_loss: 0.4858 - val_accuracy: 0.8778
Epoch 5/50
10000/10000 [==============================] - 26s 3ms/step - loss: 0.3342 - accuracy: 0.9240 - val_loss: 0.3463 - val_accuracy: 0.9160
Epoch 6/50
10000/10000 [==============================] - 26s 3ms/step - loss: 0.2417 - accuracy: 0.9445 - val_loss: 0.3149 - val_accuracy: 0.9206
Epoch 7/50
10000/100

In [58]:
# テストデータで予測
pred_test_1 = model_1.predict_classes(X_test)

In [59]:
pred_test_1[:5]

array([19, 52, 27, 63, 18], dtype=int64)

In [60]:
accuracy_score(y_eval, pred_test_1)

0.9502

In [64]:
# モデル２
model_2 = Sequential()

# 畳み込み層
model_2.add(Conv2D(32, kernel_size=(3, 3), 
                  activation='relu',
                  input_shape=(48, 48, 1)))
# プール層
model_2.add(MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト
model_2.add(Dropout(0.25))

# 畳み込み層
model_2.add(Conv2D(64, kernel_size=(3, 3), 
                  activation='relu'))
# プール層
model_2.add(MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト
model_2.add(Dropout(0.25))

# 全結合層
model_2.add(Flatten())
model_2.add(Dense(128, activation='relu'))
model_2.add(Dense(73, activation='softmax'))


In [66]:
# モデルコンパイル
model_2.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [67]:
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6400)             

In [68]:
# モデル２で訓練
model_2.fit(X_train, y_train, validation_data=(X_test, y_tset), callbacks=[callbacks], epochs=50)

Train on 10000 samples, validate on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 35s 4ms/step - loss: 4.2629 - accuracy: 0.0209 - val_loss: 4.2459 - val_accuracy: 0.0344
Epoch 2/50
10000/10000 [==============================] - 32s 3ms/step - loss: 4.0988 - accuracy: 0.0589 - val_loss: 3.8901 - val_accuracy: 0.0890
Epoch 3/50
10000/10000 [==============================] - 31s 3ms/step - loss: 2.8804 - accuracy: 0.2926 - val_loss: 2.1039 - val_accuracy: 0.4416
Epoch 4/50
10000/10000 [==============================] - 31s 3ms/step - loss: 1.0708 - accuracy: 0.7240 - val_loss: 0.8195 - val_accuracy: 0.7921
Epoch 5/50
10000/10000 [==============================] - 32s 3ms/step - loss: 0.5654 - accuracy: 0.8618 - val_loss: 0.4723 - val_accuracy: 0.9011
Epoch 6/50
10000/10000 [==============================] - 32s 3ms/step - loss: 0.4399 - accuracy: 0.8879 - val_loss: 0.4427 - val_accuracy: 0.8970
Epoch 7/50
10000/10000 [==============================] - 31s 3ms/st

In [69]:
# テストデータで予測
pred_test_2 = model_2.predict_classes(X_test)

In [70]:
pred_test_2[:5]

array([19, 52, 27, 63, 18], dtype=int64)

In [71]:
accuracy_score(y_eval, pred_test_2)

0.9522

In [72]:
# モデル３
model_3 = Sequential()

# 畳み込み層
model_3.add(Conv2D(32, kernel_size=(3, 3), 
                  activation='relu',
                  input_shape=(48, 48, 1)))
# プール層
model_3.add(MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト
model_3.add(Dropout(0.25))

# 畳み込み層
model_3.add(Conv2D(64, kernel_size=(3, 3), 
                  activation='relu'))
# プール層
model_3.add(MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト
model_3.add(Dropout(0.25))

# 畳み込み層
model_3.add(Conv2D(128, kernel_size=(3, 3), 
                  activation='relu'))
# プール層
model_3.add(MaxPooling2D(pool_size=(2, 2)))
# ドロップアウト
model_3.add(Dropout(0.25))

# 全結合層
model_3.add(Flatten())
model_3.add(Dense(128, activation='relu'))
model_3.add(Dense(73, activation='softmax'))


In [73]:
# モデルコンパイル
model_3.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [74]:
model_3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 128)        

In [75]:
# モデル３で訓練
model_3.fit(X_train, y_train, validation_data=(X_test, y_tset), callbacks=[callbacks], epochs=50)

Train on 10000 samples, validate on 10000 samples
Epoch 1/50
10000/10000 [==============================] - 44s 4ms/step - loss: 4.2828 - accuracy: 0.0164 - val_loss: 4.2774 - val_accuracy: 0.0157
Epoch 2/50
10000/10000 [==============================] - 40s 4ms/step - loss: 4.2584 - accuracy: 0.0204 - val_loss: 4.2561 - val_accuracy: 0.0157
Epoch 3/50
10000/10000 [==============================] - 40s 4ms/step - loss: 4.2114 - accuracy: 0.0332 - val_loss: 4.1985 - val_accuracy: 0.0774
Epoch 4/50
10000/10000 [==============================] - 40s 4ms/step - loss: 3.9420 - accuracy: 0.0842 - val_loss: 3.6175 - val_accuracy: 0.1589
Epoch 5/50
10000/10000 [==============================] - 43s 4ms/step - loss: 2.3299 - accuracy: 0.4059 - val_loss: 1.3149 - val_accuracy: 0.7241
Epoch 6/50
10000/10000 [==============================] - 42s 4ms/step - loss: 0.8999 - accuracy: 0.7582 - val_loss: 0.6350 - val_accuracy: 0.8561
Epoch 7/50
10000/10000 [==============================] - 41s 4ms/st

In [76]:
# テストデータで予測
pred_test_3 = model_3.predict_classes(X_test)

In [77]:
pred_test_3[:5]

array([19, 52, 27, 63, 18], dtype=int64)

In [78]:
accuracy_score(y_eval, pred_test_3)

0.9584